# Ch.13 パフォーマンスの測定とオーダー記法

## 13.1 timeitモジュール

- XOR（排他的論理和）
  - `^`ビット演算子を使う
- ランタイム
  - プログラムやコードの実行時間
- ラインタイム時のエラー
  - プログラムが実行しているときに発生したエラー

In [1]:
# XORアルゴリズム
# 一時変数を使わずに、2つの整数値を入れ替える
a, b = 42, 101
print(a, b)
# XOR演算
a = a ^ b
b = a ^ b
a = a ^ b
print(a, b)

42 101
101 42


In [8]:
# XORアルゴリズムのランタイムを測定する
import timeit

timeit.timeit(
    'a, b = 42, 101; a = a ^ b; b = a ^ b; a = a ^ b'
)

0.17749016400000528

In [14]:
code = """
a, b = 42, 101 
a = a ^ b 
b = a ^ b 
a = a ^ b
"""
timeit.timeit(code)


0.17654987100002018

In [15]:
# 一時変数を使って入れ替える
timeit.timeit(
"""a, b = 42, 101
temp = a
a = b
b =temp"""
)

0.045358574000033514

In [17]:
# アンパック
# iterable unpacking
timeit.timeit(
"""a, b = 42, 101
a, b = b, a"""
)

0.037194651999925554

In [18]:
# randomモジュールから
# 1 ~ 100までの10_000_000個の乱数を生成する
timeit.timeit(
    'random.randint(1, 100)',
    'import random',
    number=10_000_000
)

7.088929824999923

In [20]:
# timeit.timeit()に渡した文字列コードは、
# プログラムのほかの部分の変数や関数にアクセスできない

# エラーになる
# spam = 'hello'
# timeit.timeit('print(spam)', number=1)

In [21]:
spam = 'hello'
timeit.timeit('print(spam)', number=1, globals=globals())

hello


0.00035724100007428206

プログラムが動くようになってから、より効率的なプログラムを作ることに集中する

## 13.2 cProfileプロファイラー

- プロファイリング
  - プログラムの速度やメモリー使用量などを体系的に分析すること
- `cProfile`モジュール
  - Pythonのプロファイラー（プロファイリングを行うソフトウェア）

In [22]:
# 1 ~ 1_000_000までのすべての数字を合計する
import time
import cProfile

def addUpNumbers():
    total = 0
    for i in range(1, 1_000_001):
        total += i

cProfile.run('addUpNumbers()')

         4 function calls in 0.063 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.063    0.063    0.063    0.063 514933858.py:5(addUpNumbers)
        1    0.000    0.000    0.063    0.063 <string>:1(<module>)
        1    0.000    0.000    0.063    0.063 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




- ncalls
  - その関数が呼ばれた回数
- tottime
  - ほかの関数呼び出しを除いた、その関数の総実行時間
- percall
  - tottimeをncallで割ったもの
- cumtime
  - ほかの関数呼び出しを含んだ、その関数の総実行時間
- percall
  - cumtimeをncallで割ったもの
- filename:lineno(function)
  - その関数がどのファイルの何行目にあるか

- https://nostarch.com/crackingcodes
  - rsaCipher.py
  - al_sweigart_pubkey.txt
  - al_sweigart_pubkey
- 「Pythonでいかにして暗号を破るか」（ソシム、2020年）

- アムダールの法則
- タスク全体の高速化率 = 1 / ((1 - p) + (p / s))
  - s: 構成要素に加えられた高速化の割合
  - p: プログラム全体に占めるその構成要素の割合

In [27]:
import cProfile
import rsaCipher

cProfile.run(
    "rsaCipher.encryptAndWriteToFile('encrypted_file.txt', 'al_sweigart_pubkey.txt', 'abc'*100000)"
)


         11748 function calls in 34.725 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   34.725   34.725 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 _bootlocale.py:33(getpreferredencoding)
        1    0.000    0.000    0.000    0.000 codecs.py:186(__init__)
        1    0.000    0.000    0.000    0.000 codecs.py:260(__init__)
        1    0.000    0.000    0.000    0.000 codecs.py:309(__init__)
        1    0.000    0.000    0.000    0.000 codecs.py:319(decode)
        1    0.017    0.017   34.725   34.725 rsaCipher.py:104(encryptAndWriteToFile)
        1    0.254    0.254    0.256    0.256 rsaCipher.py:36(getBlocksFromText)
        1    0.013    0.013   34.705   34.705 rsaCipher.py:70(encryptMessage)
        1    0.000    0.000    0.001    0.001 rsaCipher.py:94(readKeyFile)
        1    0.000    0.000    0.000    0.000 {built-in method _codecs.utf_8_decode}
        2    

## 13.3 オーダー記法について